<a href="https://colab.research.google.com/github/mcg25035/Using-artificial-intelligence-to-crack-webpage-CAPTCHA/blob/main/OKSelenium.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 安裝套件

In [12]:
!add-apt-repository ppa:alex-p/tesseract-ocr-devel -y
!apt update
!apt install chromium-chromedriver
!apt install tesseract-ocr
!apt install tesseract-ocr-eng
!pip install selenium
!pip install Pillow
!pip install matplotlib
!pip install numpy
!pip install opencv-python
!pip install pytesseract
!mkdir results

Repository: 'deb https://ppa.launchpadcontent.net/alex-p/tesseract-ocr-devel/ubuntu/ jammy main'
Description:
This PPA contains an OCR engine - libtesseract and a command line program - tesseract. The development version available here (currntly 5.0.0 ) is better in many aspects (functionality, speed, stability) but is not 100 % API compatible with version 4.0. Tesseract 4 added a new neural net (LSTM) based OCR engine which is focused on line recognition, but also still supports the legacy Tesseract OCR engine of Tesseract 3 which works by recognizing character patterns. Compatibility with Tesseract 3 is enabled by using the Legacy OCR Engine mode (--oem 0). It also needs traineddata files which support the legacy engine, for example those from the tessdata repository. Tesseract has unicode (UTF-8) support, and can recognize more than 100 languages "out of the box". Tesseract supports various output formats: plain text, hOCR (HTML), PDF, invisible-text-only PDF, TSV. The master branch

# 套件引入

In [2]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from PIL import Image
from google.colab.patches import cv2_imshow
import matplotlib.pyplot as plt
import numpy as np
import cv2
import os
import pytesseract
import time


# 文字辨識函式



In [4]:
# convert image to char
def ocr_char(img):
    # cv2_imshow(img)
    tess_config = r'--oem 3 --psm 7 -c tessedit_char_whitelist=o123456789abcdefg'
    char = pytesseract.image_to_string(image=img, config=tess_config)
    char = char.replace("\n","").replace(" ","").replace("o","0")
    print(char)
    if len(char) == 0:
        char = "8"
    return char

# 裁切、去雜訊函式

In [5]:
# reduce noises in image
def denoise_img(img):
    img = img.astype(float) / 255
    brightness = 1 - np.max(img, axis=2)
    brightness = (255 * brightness).astype(np.uint8)
    binaryThresh = 230
    # image_show_debug("test", brightness)
    filtered = cv2.threshold(brightness, binaryThresh, 255, cv2.THRESH_BINARY)[1]
    # image_show_debug("test", filtered)
    return filtered

In [6]:
# split the image by chars
def split_img(img):
	img = img[20:35,49:160]
	x_px_indx = 1
	result = []
	for i in range(1,7):
		result.append(img[2:16, x_px_indx:x_px_indx+9])
		x_px_indx += 20
	return result

# 總解碼函式

In [7]:
# convert image to string
def decode(captcha_img):
    # image_show_debug("test", captcha_img)
    result = ""
    splited_text_img = split_img(denoise_img(captcha_img))
    for i in splited_text_img:
        result += ocr_char(i)
    return result

# 主程式

瀏覽器初始化

In [88]:
# initalizing the browser
service = Service()
options = webdriver.ChromeOptions()
options.add_argument('--headless')
options.add_argument('--no-sandbox')
options.add_argument('--disable-dev-shm-usage')
driver = webdriver.Chrome(service=service, options=options)
driver.get("http://siliconexam.com/st-register.php")

取得captcha影像

In [89]:
# getting the captcha image
w, h = 800,700
driver.set_window_size(w, h)
driver.execute_script("window.scrollTo(0,document.body.scrollHeight);")
document_height = driver.execute_script("return document.body.scrollHeight")
driver.save_screenshot('bottom.png')

captchaimage = driver.find_element(By.CLASS_NAME,"tp")
captchaimage = driver.find_element(By.XPATH,'//img[@id="chkimg"]')

left = captchaimage.location['x']
right = left + captchaimage.size['width']
top = h - (document_height - captchaimage.location['y'])
bottom = top + captchaimage.size['height']

存成圖片檔(之後給opencv讀寫)

In [90]:
# save to png file
img = Image.open('bottom.png')
captcha = img.crop((left, top, right, bottom))
captcha.save('captcha.png')


套用圖片辨識函式

In [91]:
# appling ocr functions
img = cv2.imread('captcha.png')
result = decode(img)

d


2


2


f


2


f


填寫回網站

In [92]:
# write back to website
input_element = driver.find_element(By.NAME,"rvcode")
input_element.send_keys(result)

儲存執行結果

In [93]:
# save tests result to folder
file_suffix = -1
pathtofile = ""
while True:
    file_suffix += 1
    pathtofile = f"results/test{str(file_suffix)}.png"
    if (not os.path.exists(pathtofile)):
        break

driver.save_screenshot(pathtofile)

True